In [1]:
using Revise
using BenchmarkTools

In [2]:
using DataIO
using JSON

using GCTGMT
using Support

using FeatureSetEnrichment

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7420203633609088640\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15322866291382990487\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7420203633609088640\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
benchmarks_directory_path = "/Users/kwat/Downloads/score_and_set/"

benchmark_ = []

method_ = []

benchmark_x_method = []

Any[]

In [ ]:
for name in readdir(benchmarks_directory_path)[1:5]
    
    if name[1] == '.'

        continue
        
    end

    benchmark_directory_path = joinpath(benchmarks_directory_path, name)

    element_, score_ = eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = convert(Dict{String, Vector{String}}, JSON.parse(open(joinpath(benchmark_directory_path, "gene_set.json"))))

    gmt_path_ = map(gmt_path -> replace(gmt_path, ".." => "/Users/kwat/Downloads/"), json_dict["gene_sets_tested"])

    set_to_element_ = read_gmt(gmt_path_)

    set_ = collect(keys(set_to_element_))

    n_set = length(set_)

    set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

    # Todo: get method only once at the top
    method_ = collect(keys(collect(values(set_to_method_to_result))[1]))

    set_x_method = Matrix(undef, n_set, length(method_))

    for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

        set_x_method_row = []

        for result in values(method_to_result)

            append!(set_x_method_row, result[2])

        end

        set_x_method[set_i, :] = set_x_method_row

    end

    for set in json_dict["gene_sets_positive"]

        push!(benchmark_, "$name.$set")

        benchmark_x_method_row = []

        for (method_i, set_score_) in enumerate(eachcol(set_x_method))

            set_score_, set_ = sort_like((Float64.(set_score_), set_))

            push!(benchmark_x_method_row, findfirst(set_ .== set) / n_set)

        end

        push!(benchmark_x_method, Float64.(benchmark_x_method_row))

    end

end

In [ ]:
benchmark_x_method